In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import  train_test_split
import matplotlib.pyplot as plt

In [2]:
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    for i in range(len(data)):
        if data[i,-1] == 0:
            data[i,-1] = -1
    # print(data)
    return data[:,:2], data[:,-1]

In [161]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=13)

In [162]:
class SVM_SMO():
    def __init__(self,max_iter=1000,C=1.0,kernel='linear',c=0.5,d=3,b=0):
        self.C=C
        self.max_iter=max_iter
        self.kernel=kernel
        self.c=c
        self.d=d
        self.b=b
    
    def fit(self,x,y):
        self.x=x
        self.y=y
        self.m,self.n=x.shape
        self.alpha=[0.0]*self.m
        self.E=np.array([self._E_(i) for i in range(self.m)])
        print(self.E)
        self._smo()
        print(self.alpha)
        
    ######## init E ,e=gx-y
    
    # gx:sigma a,y,k(x,xi) b
    def _g_(self,i):
        k=[self._k_(i,j) for j in range(self.m)]
#         print(k)
        return np.dot(self.alpha*self.y,k)+self.b
    
    # k:kernel func
    def _k_(self,i,j):
        if isinstance(i,int):
            if self.kernel == 'linear':
                return np.inner(self.x[i],self.x[j])
            elif self.kernel=='polynomial':
                return (np.dot(self.x[i],self.x[j])+self.c)**self.d
        else:
            if self.kernel == 'linear':
                return np.inner(i,self.x[j])
            elif self.kernel=='polynomial':
                return (np.dot(i,self.x[j])+self.c)**self.d 
    
    def _E_(self,i):
#         print(self._g_(i))
        return self._g_(i)-self.y[i]
    
    def _satisfy_kkt_(self,i):
        tmp=self.y[i]*self._g_(i)
        if self.alpha[i]==0:
            return tmp>=1
        elif self.alpha[i]>self.C:
            return tmp<=1
        else:
            return tmp==1
        
    
    def _select_2_(self):
        alpha_1_index=[i for i in range(self.m) if self.alpha[i]>0 and self.alpha[i]<self.C]
        alpha_2_index=list(set(list(range(self.m)))-set(alpha_1_index))
        alpha_1_index.extend(alpha_2_index)
        
        for i in alpha_1_index:
            if self._satisfy_kkt_(i):
                continue
            imax=(0,0)
            E1=self.E[i]
            alpha_1_index.remove(i)
            for j in alpha_1_index:
                E2=self.E[j]
                if abs(E1-E2)>imax[0]:
                    imax=(abs(E1-E2),j)
            return i, imax[1]
        
    def _smo(self):
        for _ in range(self.max_iter):
            if (_+1)%100==0:
                print('iter round :%d' % (_+1))
            
            tup=self._select_2_()
            if tup is None:
                print('satisfy kkt,stop')
                break
            else:
                i1,i2=tup
            
            E1,E2=self.E[i1],self.E[i2]
            eta=self._k_(i1,i1)+self._k_(i2,i2)-2*self._k_(i1,i2)
            
            alpha2_new_unc=self.alpha[i2]+self.y[i2]*(E1-E2)/eta
            
            if self.y[i1] == self.y[i2]:
                L = max(0, self.alpha[i2] + self.alpha[i1] - self.C)
                H = min(self.C, self.alpha[i2] + self.alpha[i1])
            else:
                L = max(0, self.alpha[i2] - self.alpha[i1])
                H = min(self.C, self.C + self.alpha[i2] - self.alpha[i1])
                
            alpha2_new = H if alpha2_new_unc > H else L if alpha2_new_unc < L else alpha2_new_unc

            alpha1_new = self.alpha[i1] + self.y[i1]*self.y[i2]*(self.alpha[i2] - alpha2_new)

            # update b
            b1_new = -E1 - self.y[i1]*self._k_(i1,i1)*(alpha1_new - self.alpha[i1]) \
                     - self.y[i2]*self._k_(i2,i1)*(alpha2_new - self.alpha[i2]) + self.b
            b2_new = -E2 - self.y[i1]*self._k_(i1,i2)*(alpha1_new - self.alpha[i1]) \
                     - self.y[i2]*self._k_(i2,i2)*(alpha2_new - self.alpha[i2]) + self.b

            # update b, alpha1, alpha2, E1, E2
            if alpha1_new > 0 and alpha1_new < self.C:
                self.b = b1_new
            elif alpha2_new > 0 and alpha2_new < self.C:
                self.b = b2_new
            else:
                self.b = (b1_new + b2_new) / 2
            self.alpha[i1] = alpha1_new
            self.alpha[i2] = alpha2_new
            self.E[i1] = self._E_(i1)
            self.E[i2] = self._E_(i2)
    
    def _predict_(self, x):
        res = self.b
        for i in range(self.m):
#             print(str(x)+'---'+str(i))
#             print(self.alpha[i])
#             print(self.y[i])
#             print(self._k_(x,i))
            res += self.alpha[i]*self.y[i]*self._k_(x, i)

        return 1 if res > 0 else -1

    def predict(self, X):
        res = [self._predict_(x1) for x1 in X]
        return res
        
        


In [169]:
smo=SVM_SMO(max_iter=5)
smo.fit(X_train,y_train)

[-1. -1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1. -1. -1.  1. -1.
 -1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1. -1.  1.
  1. -1. -1. -1.  1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1.  1.  1. -1.
  1. -1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1.  1.  1.
 -1.  1. -1.]
[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [170]:
smo.predict(X_test)

[-1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1]

In [171]:
def pred_score(y_pred,y_true):
    count=0
    for i in range(len(y_true)):
        if y_pred[i]==y_true[i]:
            count+=1
    return count/len(y_pred)

In [172]:
pred_score(smo.predict(X_test),y_test)

0.96

In [167]:
print(smo.predict(X_test))

[-1, 1, 1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1]


In [168]:
print(y_test)

[-1.  1.  1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1.  1. -1. -1.  1. -1.
  1.  1. -1. -1.  1.  1. -1.]
